In [2]:
# !pip install transformers
# !pip install sentencepiece
# !pip install matplotlib
# !pip install pandas
# !pip install numpy
# !pip install torch
# !pip install wandb
# !pip install tashaphyne
# !pip install emojis

from read_write_qrcd import *
from assets import *
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import sentencepiece
from datasets import Dataset
import pandas as pd
from datasets import load_dataset
from datasets import load_dataset, load_metric
import random
from transformers import AutoTokenizer
import transformers
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch
from transformers import default_data_collator
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
from data_preprocess import *

In [3]:
passage_question_objects = read_JSONL_file('datasets/qrcd_v1.1_train.jsonl')

Loaded 710 records from datasets/qrcd_v1.1_train.jsonl
pq_id: 2:8-16_364
pq_id: 2:174-176_364
pq_id: 14:47-52_364
pq_id: 17:12-17_364
pq_id: 18:29-31_364
pq_id: 27:89-93_364
pq_id: 30:38-41_364
pq_id: 39:38-41_364
pq_id: 91:1-10_364
pq_id: 2:8-16_378
pq_id: 2:174-176_378
pq_id: 2:277-281_378
pq_id: 2:284-286_378
pq_id: 5:48-50_378
pq_id: 6:27-32_378
pq_id: 6:100-105_378
pq_id: 6:148-150_378
pq_id: 14:47-52_378
pq_id: 16:35-40_378
pq_id: 17:12-17_378
pq_id: 18:29-31_378
pq_id: 27:89-93_378
pq_id: 30:38-41_378
pq_id: 35:15-18_378
pq_id: 39:38-41_378
pq_id: 41:45-48_378
pq_id: 42:27-31_378
pq_id: 45:12-15_378
pq_id: 47:1-6_378
pq_id: 67:1-5_378
pq_id: 73:15-19_378
pq_id: 74:32-48_378
pq_id: 76:23-31_378
pq_id: 78:38-40_378
pq_id: 81:15-29_378
pq_id: 91:1-10_378
pq_id: 2:17-20_343
pq_id: 10:3-6_343
pq_id: 24:35-38_343
pq_id: 2:25-25_143
pq_id: 55:46-78_143
pq_id: 56:11-26_143
pq_id: 56:27-40_143
pq_id: 2:26-27_231
pq_id: 2:49-52_231
pq_id: 2:53-57_231
pq_id: 2:63-66_231
pq_id: 2:67-73_231


In [4]:
df_file = json_to_dataframe(passage_question_objects)

In [5]:
df_file.head()

,id,context,question,answers
0,2:8-16_364,ومن الناس من يقول امنا بالله وباليوم الاخر وما...,لماذا سيحاسب ويعذب الضال يوم القيامه ان كان من...,{'text': ['اولئك الذين اشتروا الضلاله بالهدي']...
1,2:174-176_364,ان الذين يكتمون ما انزل الله من الكتاب ويشترون...,لماذا سيحاسب ويعذب الضال يوم القيامه ان كان من...,{'text': ['اولئك الذين اشتروا الضلاله بالهدي و...
2,14:47-52_364,فلا تحسبن الله مخلف وعده رسله ان الله عزيز ذو ...,لماذا سيحاسب ويعذب الضال يوم القيامه ان كان من...,"{'text': ['ليجزي الله كل نفس ما كسبت'], 'answe..."
3,17:12-17_364,وجعلنا الليل والنهار ايتين فمحونا ايه الليل وج...,لماذا سيحاسب ويعذب الضال يوم القيامه ان كان من...,"{'text': ['كل انسان الزمناه طائره في عنقه', 'م..."
4,18:29-31_364,وقل الحق من ربكم فمن شاء فليؤمن ومن شاء فليكفر...,لماذا سيحاسب ويعذب الضال يوم القيامه ان كان من...,"{'text': ['من شاء فليؤمن ومن شاء فليكفر'], 'an..."


In [6]:
dataset = Dataset.from_pandas(df_file)

In [7]:
type(dataset)

datasets.arrow_dataset.Dataset

In [8]:
dataset[0]

{'id': '2:8-16_364',
 'context': 'ومن الناس من يقول امنا بالله وباليوم الاخر وما هم بمؤمنين . يخادعون الله والذين امنوا وما يخدعون الا انفسهم وما يشعرون . في قلوبهم مرض فزادهم الله مرضا ولهم عذاب اليم بما كانوا يكذبون . واذا قيل لهم لا تفسدوا في الارض قالوا انما نحن مصلحون . الا انهم هم المفسدون ولكن لا يشعرون . واذا قيل لهم امنوا كما امن الناس قالوا انؤمن كما امن السفهاء الا انهم هم السفهاء ولكن لا يعلمون . واذا لقوا الذين امنوا قالوا امنا واذا خلوا الي شياطينهم قالوا انا معكم انما نحن مستهزئون . الله يستهزئ بهم ويمدهم في طغيانهم يعمهون . اولئك الذين اشتروا الضلاله بالهدي فما ربحت تجارتهم وما كانوا مهتدين . ',
 'question': 'لماذا سيحاسب ويعذب الضال يوم القيامه ان كان من يضلل الله فما له من هاد كما ورد من قوله تعالي في ايه 23 و ايه 36 من سوره الزمر ؟ ',
 'answers': {'answer_start': [504],
  'text': ['اولئك الذين اشتروا الضلاله بالهدي']}}

In [9]:
len(dataset)

710

In [10]:
# datasets = load_dataset("squad")
def visualize(datasets, n_questions=10):
    n = len(datasets)
    random_questions=random.choices(list(range(n)),k=n_questions)
    
    for i in random_questions:
        print(f"Context:{datasets[i]['context']}")
        print(f"Question:{datasets[i]['question']}")
        print(f"Answer:{datasets[i]['answers']['text']}")
        print(f"Answer Start in Text:{datasets[i]['answers']['answer_start']}")
        print("-"*100)
visualize(dataset)

Context:واذكر اخا عاد اذ انذر قومه بالاحقاف وقد خلت النذر من بين يديه ومن خلفه الا تعبدوا الا الله اني اخاف عليكم عذاب يوم عظيم . قالوا اجئتنا لتافكنا عن الهتنا فاتنا بما تعدنا ان كنت من الصادقين . قال انما العلم عند الله وابلغكم ما ارسلت به ولكني اراكم قوما تجهلون . فلما راوه عارضا مستقبل اوديتهم قالوا هذا عارض ممطرنا بل هو ما استعجلتم به ريح فيها عذاب اليم . تدمر كل شيء بامر ربها فاصبحوا لا يري الا مساكنهم كذلك نجزي القوم المجرمين . 
Question:اين عاش قوم عاد ؟ 
Answer:['بالاحقاف']
Answer Start in Text:[27]
----------------------------------------------------------------------------------------------------
Context:يا ايها الذين امنوا من يرتد منكم عن دينه فسوف ياتي الله بقوم يحبهم ويحبونه اذله علي المؤمنين اعزه علي الكافرين يجاهدون في سبيل الله ولا يخافون لومه لائم ذلك فضل الله يؤتيه من يشاء والله واسع عليم . انما وليكم الله ورسوله والذين امنوا الذين يقيمون الصلاه ويؤتون الزكاه وهم راكعون . ومن يتول الله ورسوله والذين امنوا فان حزب الله هم الغالبون . 
Question:ما هي الدلائل التي تشير ب

In [11]:
# Dealing with long docs:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it

def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" ],
        examples["context" ],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    # Looks like [0,1,2,2,2,3,4,5,5...] - Here 2nd input pair has been split in 3 parts
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    # Looks like [[(0,0),(0,3),(3,4)...] ] - Contains the actual start indices and end indices for each word in the input.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    print("First===================")
    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [12]:
dataset.column_names

['id', 'context', 'question', 'answers']

In [13]:
model_checkpoint = "salti/AraElectra-base-finetuned-ARCD"

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)



Downloading:   0%|          | 0.00/771 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/514M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/390 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/743k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [14]:
tokenized_datasets = dataset.map(prepare_train_features, batched=True, remove_columns=dataset.column_names)


  0%|          | 0/1 [00:00<?, ?ba/s]

First===================


In [15]:
tokenized_datasets['input_ids'][0]

[2,
 3346,
 20120,
 1006,
 225,
 1845,
 2552,
 41902,
 759,
 4019,
 195,
 338,
 418,
 306,
 7641,
 3986,
 647,
 3671,
 595,
 306,
 9237,
 547,
 3663,
 306,
 4151,
 55497,
 305,
 14071,
 2474,
 139,
 14071,
 4337,
 306,
 16398,
 195,
 33992,
 105,
 3,
 623,
 1571,
 306,
 1721,
 5330,
 181,
 10515,
 2609,
 883,
 201,
 11356,
 1177,
 1891,
 28076,
 9133,
 20,
 2186,
 47787,
 319,
 647,
 6767,
 5330,
 330,
 1177,
 2186,
 36288,
 319,
 335,
 15265,
 1177,
 16391,
 20,
 305,
 25041,
 3524,
 34968,
 22890,
 647,
 41924,
 28825,
 34064,
 11763,
 1199,
 2476,
 35029,
 319,
 20,
 6762,
 10007,
 1493,
 391,
 52708,
 330,
 305,
 6327,
 9466,
 9448,
 2006,
 28231,
 319,
 20,
 335,
 5014,
 1891,
 32758,
 5326,
 847,
 391,
 16391,
 20,
 6762,
 10007,
 1493,
 5330,
 330,
 547,
 5330,
 1571,
 9466,
 338,
 11148,
 547,
 5330,
 1512,
 2180,
 335,
 5014,
 1891,
 1512,
 2180,
 847,
 391,
 20132,
 20,
 6762,
 25642,
 860,
 5330,
 330,
 9466,
 5330,
 181,
 6762,
 28615,
 181,
 487,
 45772,
 6571,
 336,
 9466

In [16]:

args = TrainingArguments(
    f"test-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

data_collator = default_data_collator
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets,
#     eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()
trainer.save_model(trainer.save_model("test-squad-trained"))

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:145: UserWarning: 
NVIDIA GeForce RTX 3070 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA GeForce RTX 3070 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 712
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
!pip install wandb


In [ ]:
!wandb login